In [12]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import complextorch as cvtorch
from torch.utils.data import DataLoader
import torch
from torch.utils.tensorboard import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

import matplotlib.pyplot as plt
import json
import os
from collections import defaultdict
import random
import re

import antiglitch
from antiglitch import to_fd

Using device cuda


In [13]:
train_complex = True
train_real = True

## Data processing

In [30]:
rootdir = '/home/xangma/OneDrive/repos/antiglitch/'
datadir = rootdir + 'data/'

In [15]:
# Load data
# files are named as follows: ifo-key-num.npz,
# e.g. L1-lowblip-0355.npz

# Find all ifos, keys, and numbers in the data directory
ifos = []
ml_models = []
numbers = []

# filename pattern: ifo-key-num.npz
filename_pattern = re.compile(r'([A-Z0-9]+)-([a-z]+)-(\d+).npz')

for file in os.listdir(datadir):
	# Get unique ifos:
	try:
		# check filename format
		if filename_pattern.match(file):
			ifo = file.split('-')[0]
			if ifo not in ifos:
				ifos.append(ifo)
			# Get unique keys:
			ml_model = file.split('-')[1]
			if ml_model not in ml_models:
				ml_models.append(ml_model)
	except:
		pass

datadict={}
for ifo in ifos:
	datadict[ifo] = {}
	for ml_model in ml_models:
		datadict[ifo][ml_model] = []
		for file in os.listdir(datadir):
			try:
				ifo_file = file.split('-')[0]
				key_file = file.split('-')[1]
				num = file.split('-')[2].split('.')[0]
				if ifo_file == ifo and key_file == ml_model:
					datadict[ifo][ml_model].append(int(num))
			except:
				pass

In [16]:
# load results
resultsjson=datadir + 'all_PE_v3.json'
with open(resultsjson, 'r') as f:
	results = json.load(f)

In [17]:
# load all glitches into an array using the datadict and Snippet class
glitches = {}
for ifo in ifos:
    if ifo not in glitches:
        glitches[ifo] = {}
    for ml_model in ml_models:
        if ml_model not in glitches[ifo]:
            glitches[ifo][ml_model] = {}
        for num in datadict[ifo][ml_model]:
            # try:
                if num not in glitches[ifo][ml_model]:
                    glitches[ifo][ml_model][num] = {}
                snip = antiglitch.SnippetNormed(ifo, ml_model, num, datadir)
                glitches[ifo][ml_model][num]['data'] = to_fd(snip.whts)
                glitches[ifo][ml_model][num]['invasd'] = snip.invasd
            # except:
            #     pass

In [18]:
ml_label_map = {'Blip_Low_Frequency':'lowblip', 'Blip':'blip', 'Koi_Fish':'koi', 'Tomte':'tomte'}
results_keys = ['f0', 'f0_sd', 'gbw', 'gbw_sd', 'amp_r', 'amp_r_sd', 'amp_i', 'amp_i_sd', 'time', 'time_sd', 'num']

nums = list(results['num'].values())
for i in range(len(nums)):
    try:
        stri = str(i)
        ifo = results['ifo'][stri]
        ml_label = ml_label_map[results['ml_label'][stri]]
        
        for key in results_keys:
            glitches[ifo][ml_label][nums[i]][key] = results[key][stri]
    except:
        pass

In [19]:
# Step 1: Restructure the dataset (Yes I know it was originally in this format, you leave me alone)
distributions = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for ifo in glitches:
    for ml_model in glitches[ifo]:
        for glitch_num in glitches[ifo][ml_model]:
            for key in results_keys:
                distributions[ifo][ml_model][key].append(glitches[ifo][ml_model][glitch_num][key])

In [20]:
def gen_sample(ifo, ml_model, tosample=['f0', 'gbw', 'amp_r', 'amp_i', 'time']):
    res = {key:None for key in tosample}
    for key in tosample:
        draw = np.random.choice(distributions[ifo][ml_model][key])
        draw_sd = distributions[ifo][ml_model][key + '_sd'][distributions[ifo][ml_model][key].index(draw)]
        draw_final = np.random.normal(draw, draw_sd)
        res[key] = draw_final
    return res

def new_init_Snippet(self, invasd):
        self.invasd = invasd

SnippetNormed = type('SnippetNormed', (antiglitch.SnippetNormed,), {'__init__': new_init_Snippet})


In [21]:
def get_snip(ifos, ml_models, tosample=['f0', 'gbw', 'amp_r', 'amp_i', 'time']):
		ifo = np.random.choice(ifos)
		ml_model = np.random.choice(ml_models)
		glitch_num = random.choice(datadict[ifo][ml_model])
		glitch_invasd = glitches[ifo][ml_model][glitch_num]['invasd']
		snip = SnippetNormed(glitch_invasd)
		inf = gen_sample(ifo, ml_model)
		inf['freqs'] = snip.invasd
		snip.set_infer(inf)
		x = snip.fglitch
		# check for nans
		if np.isnan(x).any():
			return get_snip(ifos, ml_models, tosample)
		y = (snip.inf['f0'], snip.inf['gbw'])
		return x, y

In [22]:
# define dataset
NTRAIN = 200000
NTEST = 50000
cachedataset_prefix = "antiglitch_cvnn_dataset_"

class GlitchDataset(torch.utils.data.TensorDataset):
	def __init__(self, ifos, ml_models, datadict, glitches, distributions, size, tr_scalings=None):
		self.ifos = ifos
		self.ml_models = ml_models
		self.datadict = datadict
		self.glitches = glitches
		self.distributions = distributions
		self.size = size
		self.x_arr = []
		self.y_arr = []
		if tr_scalings != None:
			self.x_mean_real, self.x_std_real, self.x_mean_imag, self.x_std_imag, self.y1_mean, self.y1_std, self.y2_mean, self.y2_std = tr_scalings
		# create dataset
		if cachedataset_prefix + str(self.size) + '.npz' in os.listdir(datadir):
			data = np.load(datadir + cachedataset_prefix + str(self.size) + '.npz')
			self.x_arr = data['x_arr']
			self.y_arr = data['y_arr']
			if tr_scalings == None:
				self.x_mean_real = np.mean(self.x_arr.real)
				self.x_std_real = np.std(self.x_arr.real)
				
				self.x_mean_imag = np.mean(self.x_arr.imag)
				self.x_std_imag = np.std(self.x_arr.imag)
				self.y1_mean = self.y_arr[:,0].mean()
				self.y1_std = self.y_arr[:,0].std()
				self.y2_mean = self.y_arr[:,1].mean()
				self.y2_std = self.y_arr[:,1].std()
		else:
			for i in range(size):
				x, y = get_snip(ifos, ml_models)
				self.x_arr.append(x)
				self.y_arr.append(y)
			self.x_arr = np.array(self.x_arr)
			self.y_arr = np.array(self.y_arr)
			# normalize x_arr
			real_parts = np.real(self.x_arr)
			imag_parts = np.imag(self.x_arr)
			
			if tr_scalings == None:
				self.x_mean_real = np.mean(real_parts)
				self.x_std_real = np.std(real_parts)
				
				self.x_mean_imag = np.mean(imag_parts)
				self.x_std_imag = np.std(imag_parts)
			
			normalized_real = (real_parts - self.x_mean_real) / self.x_std_real
			normalized_imag = (imag_parts - self.x_mean_imag) / self.x_std_imag
			
			self.x_arr = normalized_real + 1j * normalized_imag
			# normalize y_arr
			if tr_scalings == None:
				self.y1_mean = self.y_arr[:,0].mean()
				self.y1_std = self.y_arr[:,0].std()
				self.y2_mean = self.y_arr[:,1].mean()
				self.y2_std = self.y_arr[:,1].std()
			self.y_arr[:,0] = (self.y_arr[:,0] - self.y1_mean)/self.y1_std
			self.y_arr[:,1] = (self.y_arr[:,1] - self.y2_mean)/self.y2_std
			np.savez(datadir + cachedataset_prefix + str(size) + '.npz', x_arr=self.x_arr, y_arr=self.y_arr)
		# convert to tensors
		self.x_arr = torch.tensor(self.x_arr, dtype=torch.complex64, device=device)
		self.y_arr = torch.tensor(self.y_arr, dtype=torch.float32, device=device)

	def __len__(self):
		return self.size

	def __getitem__(self, idx):
		return self.x_arr[idx], self.y_arr[idx]

train_data = GlitchDataset(ifos, ml_models, datadict, glitches, distributions, NTRAIN)
tr_scalings = [train_data.x_mean_real, train_data.x_std_real, train_data.x_mean_imag, train_data.x_std_imag, train_data.y1_mean, train_data.y1_std, train_data.y2_mean, train_data.y2_std]
test_data = GlitchDataset(ifos, ml_models, datadict, glitches, distributions, NTEST, tr_scalings=tr_scalings)
testitem = test_data.__getitem__(0)
print(testitem[0][1:20], testitem[1])

tensor([ 0.0038+0.0035j, -0.0185+0.0791j, -0.1046+0.3551j, -0.2490+0.7901j,
        -0.3985+1.2057j, -0.4960+1.4394j, -0.5424+1.5138j, -0.6074+1.6317j,
        -0.6671+1.7271j, -0.6741+1.6844j, -0.6812+1.6439j, -0.7071+1.6490j,
        -0.7051+1.5906j, -0.6782+1.4812j, -0.6639+1.4043j, -0.6509+1.3341j,
        -0.6241+1.2404j, -0.6020+1.1607j, -0.5896+1.1031j], device='cuda:0') tensor([-0.2473, -0.7574], device='cuda:0')


In [23]:
# These were cells used for preprocessing the data before I was told I could use the Snippet class to generate - whoops.
# # Now flatten to x and y data where x is glitched data and y is the [f0, gbw]

# x = []
# y = []
# for ifo in ifos:
# 	for key in keys:
# 		for num in datadict[ifo][key]:
# 			try:
# 				x.append(glitches[ifo][key][num]['data'])
# 				y.append([glitches[ifo][key][num]['f0'], glitches[ifo][key][num]['gbw']])
# 			except:
# 				pass
# x = np.array(x)
# y = np.array(y)
# type(x), x.shape, type(y), y.shape
# # Split data into training and testing

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# # Convert to torch tensors
# x_train = torch.tensor(x_train, dtype=torch.complex64)
# x_test = torch.tensor(x_test, dtype=torch.complex64)
# y_train = torch.tensor(y_train, dtype=torch.float32)
# y_test = torch.tensor(y_test, dtype=torch.float32)
# # Normalize data
# x_train = (x_train - x_train.mean()) / x_train.std()
# x_test = (x_test - x_test.mean()) / x_test.std()
# y_train = (y_train - y_train.mean()) / y_train.std()
# y_test = (y_test - y_test.mean()) / y_test.std()
# # put data on device
# x_train = x_train.to(device)
# x_test = x_test.to(device)
# y_train = y_train.to(device)
# y_test = y_test.to(device)
# # Print out a description and sample of the data
# print(x_train.shape, x_train.dtype, x_train[0])
# print(x_test.shape, x_test.dtype, x_test[0])
# print(y_train.shape, y_train.dtype, y_train[0])
# print(y_test.shape, y_test.dtype, y_test[0])
# train_data = torch.utils.data.TensorDataset(x_train, y_train)
# test_data = torch.utils.data.TensorDataset(x_test, y_test)

In [24]:
# Load into DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=1024, shuffle=True, drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=1024, shuffle=False, drop_last=False)
list(train_loader.__iter__())[0][0].shape

torch.Size([1024, 513])

## Complex Model Definition

In [25]:
class ComplexValuedNN(nn.Module):
    def __init__(self):
        super(ComplexValuedNN, self).__init__()

        # Complex Conv Layer 1
        self.conv1 = cvtorch.nn.CVConv1d(1, 32, 3, padding="same")
        self.activation1 = cvtorch.nn.CVCardiod()
        # self.bn1 = cvtorch.nn.CVBatchNorm1d(32)
        # self.dropout1 = cvtorch.nn.CVDropout(0.25)

        # Complex Conv Layer 2
        self.conv2 = cvtorch.nn.CVConv1d(32, 64, 5, padding="same")
        self.activation2 = cvtorch.nn.CVCardiod()
        # self.bn2 = cvtorch.nn.CVBatchNorm1d(64)
        # self.dropout2 = cvtorch.nn.CVDropout(0.25)

        # Fully connected Layer 1
        self.fc1 = cvtorch.nn.CVLinear(64 * 513, 128)
        self.activation3 = cvtorch.nn.CVCardiod()
        # self.bn3 = cvtorch.nn.CVBatchNorm1d(1024)
        # self.dropout3 = cvtorch.nn.CVDropout(0.25)

        # Fully connected Layer 2
        # self.fc2 = cvtorch.nn.CVLinear(1024, 1024)
        # self.activation4 = cvtorch.nn.zReLU()
        # self.bn4 = cvtorch.nn.CVBatchNorm1d(1024)
        # self.dropout4 = cvtorch.nn.CVDropout(0.25)
        # Fully connected Layer 2
        self.fc3 = cvtorch.nn.CVLinear(128, 32)
        self.activation5 = cvtorch.nn.CVCardiod()
        
        # self.bn5 = cvtorch.nn.CVBatchNorm1d(32)
        # self.dropout5 = cvtorch.nn.CVDropout(0.25)
        
        # Output Layer (real-valued)
        self.output_layer = nn.Linear(32 * 2, 2)

    def forward(self, x):
        # Add channel dimension for convolution
        x = x.unsqueeze(1)

        # Conv Layer 1
        x = self.conv1(x)
        x = self.activation1(x)
        # x = self.bn1(x)
        # x = self.dropout1(x)

        # # # Conv Layer 2
        x = self.conv2(x)
        x = self.activation2(x)
        # x = self.bn2(x)
        # x = self.dropout2(x)

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Fully Connected Layer 1
        x = self.fc1(x)
        x = self.activation3(x)
        # x = self.bn3(x)
        # x = self.dropout3(x)

        # Fully Connected Layer 2
        # x = self.fc2(x)
        # x = self.activation4(x)
        # x = self.bn4(x)
        # x = self.dropout4(x)
        x = self.fc3(x)
        x = self.activation5(x)
        # x = self.bn5(x)
        # x = self.dropout5(x)
        # Transform complex valued output for the final real-valued layer
        real_x = torch.cat((x.real, x.imag), dim=1)  # Merge real and imag parts

        # Output Layer
        out = self.output_layer(real_x)

        return out


# Real Model Definition

In [26]:
class RealValuedNN(nn.Module):
    def __init__(self):
        super(RealValuedNN, self).__init__()

        # Conv Layer 1
        self.conv1_real = nn.Conv1d(1, 32, kernel_size=3, padding='same')
        self.conv1_imag = nn.Conv1d(1, 32, kernel_size=3, padding='same')

        # Activation 1
        self.activation1_real = nn.ReLU()
        self.activation1_imag = nn.ReLU()

        # Conv Layer 2
        self.conv2_real = nn.Conv1d(32, 64, kernel_size=5, padding='same')
        self.conv2_imag = nn.Conv1d(32, 64, kernel_size=5, padding='same')

        # Activation 2
        self.activation2_real = nn.ReLU()
        self.activation2_imag = nn.ReLU()

        # Given input size is (512), Convolution does not alter the dimension with padding='same'
        # Therefore, the size is the same after convolutions (512).
        # Flatten the tensor
        self.flatten_size = 64 * 513  # Adjusted size to reflect conv2 output * sequence length

        # Fully connected Layer 1
        self.fc1_real = nn.Linear(self.flatten_size, 128)
        self.fc1_imag = nn.Linear(self.flatten_size, 128)

        # Activation 3
        self.activation3_real = nn.ReLU()
        self.activation3_imag = nn.ReLU()

        # Fully connected Layer 2
        self.fc2_real = nn.Linear(128, 32)
        self.fc2_imag = nn.Linear(128, 32)

        # Activation 4
        self.activation4_real = nn.ReLU()
        self.activation4_imag = nn.ReLU()
        
        # Output Layer (real-valued)
        self.output_layer = nn.Linear(32 * 2, 2)

    def forward(self, real, imag):
        # Add channel dimension for convolution
        real = real.unsqueeze(1)
        imag = imag.unsqueeze(1)

        # Conv Layer 1
        real = self.conv1_real(real)
        imag = self.conv1_imag(imag)
        real = self.activation1_real(real)
        imag = self.activation1_imag(imag)

        # Conv Layer 2
        real = self.conv2_real(real)
        imag = self.conv2_imag(imag)
        real = self.activation2_real(real)
        imag = self.activation2_imag(imag)

        # Flatten the tensors
        real = real.view(real.size(0), -1)
        imag = imag.view(imag.size(0), -1)
        
        # Fully Connected Layer 1
        real = self.fc1_real(real)
        imag = self.fc1_imag(imag)
        real = self.activation3_real(real)
        imag = self.activation3_imag(imag)

        # Fully Connected Layer 2
        real = self.fc2_real(real)
        imag = self.fc2_imag(imag)
        real = self.activation4_real(real)
        imag = self.activation4_imag(imag)

        # Concatenate the real and imaginary parts
        real_imag = torch.cat((real, imag), dim=1)

        # Output Layer
        out = self.output_layer(real_imag)

        return out

# Test the model with dummy input
model = RealValuedNN()
model.eval()
real_input = torch.randn(32, 513)  # Batch size of 32 and feature size of 1024 for real part
imag_input = torch.randn(32, 513)  # Batch size of 32 and feature size of 1024 for imaginary part
output = model(real_input, imag_input)
print(output.shape) # Expecting (32, 2)
del model, real_input, imag_input, output

torch.Size([32, 2])


## Training Loop Definition

In [36]:
# Training loop
def train(model, criterion, optimizer, scheduler, train_loader, test_loader, num_epochs, model_path, writer):
    # Load model if it exists
    try:
        if os.path.exists(model_path):
            model.load_state_dict(torch.load(model_path))
            print('Model loaded')
    except:
        print('Model not loaded')
    torch.autograd.set_detect_anomaly(True)
    for epoch in range(num_epochs):
        model.train()

        for inputs, targets in train_loader:
            # Forward pass
            if isinstance(model, ComplexValuedNN):
                outputs = model(inputs)
            elif isinstance(model, RealValuedNN):
                outputs = model(torch.real(inputs), torch.imag(inputs))
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Test the model
        model.eval()
        with torch.no_grad():
            total_loss = 0
            for inputs, targets in test_loader:
                if isinstance(model, ComplexValuedNN):
                    outputs = model(inputs)
                elif isinstance(model, RealValuedNN):
                    outputs = model(torch.real(inputs), torch.imag(inputs))
                total_loss += criterion(outputs, targets).item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {total_loss/len(test_loader):.4f}")
        
        # update the learning rate
        scheduler.step(total_loss/len(test_loader))

        writer.add_scalars("Loss", {"Train": loss.item(), "Validation": total_loss/len(test_loader)}, epoch)
        writer.add_scalar("Learning rate", optimizer.param_groups[0]['lr'], epoch)
        
        # save the model every 10 epochs
        if epoch % 10 == 0:
            torch.save(model.state_dict(), model_path)
            
    writer.flush()


## Complex Valued Training

In [39]:
# Define the input and output dimensions based on the dataset
input_dim = 513  # Number of input features
output_dim = 2  # Number of output units (as specified in the dataset)

# Create an instance of the network
model = ComplexValuedNN()

# put model on device
model.to(device)

# model = torch.compile(model)
# Print the model architecture
print(model)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)
writer = SummaryWriter("Complex_valued")
model_path = rootdir + 'notebooks/complex_nn_model.pth'

ComplexValuedNN(
  (conv1): CVConv1d(
    (conv_r): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
    (conv_i): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
  )
  (activation1): CVCardiod()
  (conv2): CVConv1d(
    (conv_r): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
    (conv_i): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
  )
  (activation2): CVCardiod()
  (fc1): CVLinear(
    (linear_r): Linear(in_features=32832, out_features=128, bias=False)
    (linear_i): Linear(in_features=32832, out_features=128, bias=False)
  )
  (activation3): CVCardiod()
  (fc3): CVLinear(
    (linear_r): Linear(in_features=128, out_features=32, bias=False)
    (linear_i): Linear(in_features=128, out_features=32, bias=False)
  )
  (activation5): CVCardiod()
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)
Number of parameters: 8434178


In [40]:
if train_complex:
    train(model, criterion, optimizer, scheduler, train_loader, test_loader, 100, model_path, writer)

Epoch [1/100], Training Loss: 0.6984, Validation Loss: 0.7925
Epoch [2/100], Training Loss: 0.8121, Validation Loss: 0.6998
Epoch [3/100], Training Loss: 0.6048, Validation Loss: 0.6318
Epoch [4/100], Training Loss: 0.4927, Validation Loss: 0.5816
Epoch [5/100], Training Loss: 0.4166, Validation Loss: 0.5386
Epoch [6/100], Training Loss: 0.6263, Validation Loss: 0.5050
Epoch [7/100], Training Loss: 0.4879, Validation Loss: 0.4704
Epoch [8/100], Training Loss: 0.4757, Validation Loss: 0.4452
Epoch [9/100], Training Loss: 0.4301, Validation Loss: 0.4200
Epoch [10/100], Training Loss: 0.4495, Validation Loss: 0.4009
Epoch [11/100], Training Loss: 0.4736, Validation Loss: 0.3825
Epoch [12/100], Training Loss: 0.2855, Validation Loss: 0.3652
Epoch [13/100], Training Loss: 0.2906, Validation Loss: 0.3493
Epoch [14/100], Training Loss: 0.2890, Validation Loss: 0.3352
Epoch [15/100], Training Loss: 0.3835, Validation Loss: 0.3211
Epoch [16/100], Training Loss: 0.2366, Validation Loss: 0.3098
E

In [41]:
if train_complex:
    # Predictions from Complex model
    # Load model
    model = ComplexValuedNN()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    # Get 10 inputs
    inputs, targets = next(iter(test_loader))
    outputs = model(inputs)

    if type(outputs) == torch.Tensor:
        outputs = outputs.cpu().detach().numpy()
    if type(targets) == torch.Tensor:
        targets = targets.cpu().detach().numpy()
    for i in range(0,10):
        print(f"Prediction: {outputs[i]}")
        print(f"Target: {targets[i]}")
        print("\n")

Prediction: [-0.36573762 -0.8493817 ]
Target: [-0.24728732 -0.75742614]


Prediction: [ 1.0960218 -0.9100221]
Target: [ 1.4674261 -0.8598122]


Prediction: [ 0.28401774 -0.946801  ]
Target: [-0.08344258 -0.94632244]


Prediction: [ 0.34813672 -0.9221524 ]
Target: [ 0.19683681 -0.8914559 ]


Prediction: [-0.48367566 -0.10606176]
Target: [-0.48456588 -0.10759523]


Prediction: [-0.39170882  1.39896   ]
Target: [-0.39610118  1.5324879 ]


Prediction: [-0.0795102   0.71615255]
Target: [0.316644 2.712674]


Prediction: [-0.50579584 -0.36977682]
Target: [-0.514367   -0.29206896]


Prediction: [-0.4589251  1.2911007]
Target: [-0.4434782  1.1970277]


Prediction: [-0.44986913 -0.35354033]
Target: [-0.5566556 -0.4382659]




# Real Valued Training

In [42]:
# Recreate the Datasets so they output [x.real, x.imag] instead of x
train_data = GlitchDataset(ifos, ml_models, datadict, glitches, distributions, NTRAIN)
test_data = GlitchDataset(ifos, ml_models, datadict, glitches, distributions, NTEST)
train_loader = DataLoader(dataset=train_data, batch_size=1024, shuffle=True, drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=1024, shuffle=False, drop_last=False)

In [43]:
input_dim = 513  # Number of input features
output_dim = 2  # Number of output units (as specified in the dataset)

# Create an instance of the network
model = RealValuedNN()

# put model on device
model.to(device)

# model = torch.compile(model)
# Print the model architecture
print(model)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, verbose=True)
writer = SummaryWriter("Real_valued")
model_path = rootdir + 'notebooks/real_nn_model.pth'

RealValuedNN(
  (conv1_real): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
  (conv1_imag): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same)
  (activation1_real): ReLU()
  (activation1_imag): ReLU()
  (conv2_real): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
  (conv2_imag): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=same)
  (activation2_real): ReLU()
  (activation2_imag): ReLU()
  (fc1_real): Linear(in_features=32832, out_features=128, bias=True)
  (fc1_imag): Linear(in_features=32832, out_features=128, bias=True)
  (activation3_real): ReLU()
  (activation3_imag): ReLU()
  (fc2_real): Linear(in_features=128, out_features=32, bias=True)
  (fc2_imag): Linear(in_features=128, out_features=32, bias=True)
  (activation4_real): ReLU()
  (activation4_imag): ReLU()
  (output_layer): Linear(in_features=64, out_features=2, bias=True)
)
Number of parameters: 8434498


In [44]:
# Training loop
if train_real:
    train(model, criterion, optimizer, scheduler, train_loader, test_loader, 100, model_path, writer)

Epoch [1/100], Training Loss: 0.7921, Validation Loss: 0.8305
Epoch [2/100], Training Loss: 0.6813, Validation Loss: 0.7392
Epoch [3/100], Training Loss: 0.7610, Validation Loss: 0.6793
Epoch [4/100], Training Loss: 0.5358, Validation Loss: 0.6317
Epoch [5/100], Training Loss: 0.6132, Validation Loss: 0.5936
Epoch [6/100], Training Loss: 0.5242, Validation Loss: 0.5638
Epoch [7/100], Training Loss: 0.5790, Validation Loss: 0.5378
Epoch [8/100], Training Loss: 0.7566, Validation Loss: 0.5163
Epoch [9/100], Training Loss: 0.4142, Validation Loss: 0.4931
Epoch [10/100], Training Loss: 0.5504, Validation Loss: 0.4744
Epoch [11/100], Training Loss: 0.4549, Validation Loss: 0.4571
Epoch [12/100], Training Loss: 0.3847, Validation Loss: 0.4407
Epoch [13/100], Training Loss: 0.3523, Validation Loss: 0.4272
Epoch [14/100], Training Loss: 0.3218, Validation Loss: 0.4125
Epoch [15/100], Training Loss: 0.4525, Validation Loss: 0.4035
Epoch [16/100], Training Loss: 0.4361, Validation Loss: 0.3886
E

In [46]:
if train_real:
    # Predictions from Real model
    # Load model
    model = RealValuedNN()
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    # Get 10 inputs
    inputs, targets = next(iter(test_loader))
    outputs = model(inputs.real, inputs.imag)

    if type(outputs) == torch.Tensor:
        outputs = outputs.cpu().detach().numpy()
    if type(targets) == torch.Tensor:
        targets = targets.cpu().detach().numpy()
    for i in range(0,10):
        print(f"Prediction: {outputs[i]}")
        print(f"Target: {targets[i]}")
        print("\n")

Prediction: [-0.38858858 -0.9720913 ]
Target: [-0.24728732 -0.75742614]


Prediction: [ 1.4360641  -0.75076455]
Target: [ 1.4674261 -0.8598122]


Prediction: [ 0.23093775 -0.9771106 ]
Target: [-0.08344258 -0.94632244]


Prediction: [ 0.55242974 -0.82873106]
Target: [ 0.19683681 -0.8914559 ]


Prediction: [-0.48066345 -0.22690825]
Target: [-0.48456588 -0.10759523]


Prediction: [-0.44693607  1.2371559 ]
Target: [-0.39610118  1.5324879 ]


Prediction: [0.02147465 0.8863181 ]
Target: [0.316644 2.712674]


Prediction: [-0.39063573 -0.00678147]
Target: [-0.514367   -0.29206896]


Prediction: [-0.46402672  1.2403691 ]
Target: [-0.4434782  1.1970277]


Prediction: [-0.3967832   0.01413908]
Target: [-0.5566556 -0.4382659]




In [ ]:
writer.